In [1]:
import mysql.connector
from nltk.tokenize import sent_tokenize
import import_ipynb
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pandas as pd
from MMR import *
from nltk.corpus import stopwords
import string
from termcolor import colored
import time

importing Jupyter notebook from MMR.ipynb


# Deklarasi Connection Database

Membuat koneksi kedatabase lokal dengan database 'db-berita-mca'

In [2]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="db-berita-mca"  
) 

# Membuat Fungsi Steaming Use Sastrawi

Membuat stemmer pakai sastrawi 
Digunakan untuk melakukan steaming kalimat (menjadi kalimat dasar Menggunakan Library Sastrawi)

In [3]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Mengambil Data pada Database

Mengambil data pada database,data yang diambil hanya 2 kolom saja

In [4]:
mycursor = mydb.cursor()
mycursor.execute("SELECT id, content FROM tb_berita")
myresult = mycursor.fetchall()

df = pd.DataFrame(myresult,columns=['id','article'])
df

,id,article
0,217,"Jakarta, CNBC Indonesia - Krisis mainan jelang..."
1,222,"Jakarta, CNBC Indonesia - Aset kripto (cryptoc..."
2,224,"Jakarta, CNBC Indonesia - Internet merupakan s..."
3,225,"Jakarta, CNBC Indonesia - Asal muasal virus co..."
4,226,"Jakarta, CNBC Indonesia - Dunia mata uang krip..."
5,227,"Jakarta, CNBC Indonesia - Staples Center akan ..."
6,228,"Jakarta, CNBC Indonesia - Tren investasi kript..."


# Load Data Stopword Lib & File Internal

Mengambil data stopword bahasa indonesia yang pertama menggunakan file list dari stop word 917 dan menggunakan library nltk sebanyak panjang data 789

In [5]:
def load_stopWords():
    r = open("stopwordlist-indonesia.txt", "r")
    readTxt = r.read()
    idStopWord = readTxt.split() 
    return idStopWord

In [6]:
def load_stopWordsNLTK():
    stop_words = stopwords.words('indonesian')
    return stop_words

# Simpan hasil summary ke database

Menyimpan data yang telah dihasilakan melalui summary ke database dengan cara mengupdate data tersebut dan memasukan text summary pada field tersebut

In [7]:
def simpanSummary(mydb,summary,id):
    mycursor = mydb.cursor()
    id = int(id)
    sql = "UPDATE tb_berita SET summary = %s WHERE id = %s"
    param = (summary,id)
    mycursor.execute(sql, param)
    mydb.commit()

# Proses Text Preprocessing 

[tokenizeParagraftoKalimat, cleansingData, steamingData, indoStopWord]

Fungsi diatas merupakan fungsi yang digunakan untuk melakukan preprocessing article sebuah berita, penjelasan mengenai fungsi akan dijelaskan lebih lanjut

[tokenizeParagraftoKalimat]

berfungsi sebagai tokenize pargraf menjadi bagian perkalimat 

In [8]:
def tokenizeParagraftoKalimat(paragraf):
    tokenize = sent_tokenize(paragraf)
    return tokenize

[cleansingData]

berfungsi sebagai membersihkan data dari huruf tanda petik dan masih kata besar atau keceil

In [9]:
def cleansingData(text):
    hasil = re.sub('[^A-Za-z0-9 ]+', '', text)
    hasil = hasil.translate(str.maketrans("","",string.punctuation))
    return hasil

[steamingData]

berguna sebagai menjadikan kata dasar sebuah kalimat dan fungsi stemmer tersebut sekaligus menjadikan kalimat menjadi lower atau huruf kecil

In [10]:
def steamingData(text):
    steamingData = stemmer.stem(text)
    return steamingData

[indoStopWord]

berguna dalam menghilangkan kata sambung atau tidak diperlukan, ini berfungsi dlam menghitung kesamaan antar kalimat pada algoritma MMR agar lebih mudah dan mendapatkan hasil yang akurat (Lib NLTK & List Stopword)

In [11]:
def indoStopWord(stopNLTK, stopTxt, kalimat):
    hasilStopWord = []
    for kata in kalimat.split():
        if not kata in stopNLTK:
            if not kata in stopTxt:
                hasilStopWord.append(kata)
    return " ".join(hasilStopWord)

[hasilSummary]

fungsi yang berguna dalam menampilkan hasil summarization pada algoritma MMR

In [12]:
def hasilSummary(kalimatAsli, summarySet):
    hasil = []
    for sentence in summarySet:
        hasil.append(kalimatAsli [sentence].lstrip(' ')) 
    return " ".join(hasil)

[visualisasiSummaryOri]

menampilkan visualisasi yang sangat berguna dalam mengetahui kata mana saja yang dipakai atau ditampilkan pada summarization

In [13]:
def visualisasiSummaryOri(cleansingTeks,summarySet,kalimatAsli):
    for sentence in cleansingTeks:
        if sentence in summarySet:
            print(colored(kalimatAsli[sentence].lstrip(' '),'red'))
        else:
            print(kalimatAsli[sentence].lstrip(' '))

bisa dibilang sebagai code utama, dimana disini tempat menjalankan semua code dengan melakukan pemanggilan semua fungsi dan class algoritam MMR 

In [14]:
for iteration, article in enumerate(df['article']):
    id = df['id'][iteration]
    tokenizeKalimat = tokenizeParagraftoKalimat(article)
    
    kalimat = []
    cleansingTeks = []
    kalimatAsli = {}

    for teks in tokenizeKalimat:
        kataDasar = steamingData(cleansingData(teks))
        dataFinal = indoStopWord(load_stopWordsNLTK(),load_stopWords(),kataDasar)
        cleansingTeks.append(dataFinal)
        kalimat.append(teks)
        kalimatAsli[dataFinal] = teks

    iteration = MMR()
    score = iteration.similarityScore(cleansingTeks)
    summarySet = iteration.calculateMMR(score,cleansingTeks)
    

    summary = hasilSummary(kalimatAsli,summarySet)
    simpanSummary(mydb,summary,id)
    print ('\nSummary dari Berita ID : {} (hasil teks yang diringkas):\n'.format(id))
    print(summary)

    print ('=============================================================')
    print ('\Oringinal Article (Teks Asli):\n')
    visualisasiSummaryOri(cleansingTeks,summarySet,kalimatAsli)


Summary dari Berita ID : 217 (hasil teks yang diringkas):

Bos retail mainan The Entertainer, Gary Grant, memperingatkan penundaan ini akan mengakibatkan kekurangan stok mainan menjelang Natal. Pelabuhan komersial terbesar di Inggris Felixstowe mengatakan kepada BBC bahwa saat ini ada 50.000 kontainer yang menunggu untuk dikumpulkan, karena kekurangan pengemudi truk HGV. Kini stok boneka Barbie dan mainan Paw Patrol, salah satu mainan favorit anak-anak, diperkirakan akan cepat habis. "Ini tantangan bagi pengecer kecil karena mereka tidak memiliki uang tunai untuk menimbun." Padahal tarif normal US$ 2.500 pada tahun 2020.
\Oringinal Article (Teks Asli):

Jakarta, CNBC Indonesia - Krisis mainan jelang Natal terjadi di Inggris.
Fenomena ini terjadi karena adanya penundaan penurunan barang-barang dari kontainer di pelabuhan Inggris.
Bos retail mainan The Entertainer, Gary Grant, memperingatkan penundaan ini akan mengakibatkan kekurangan stok mainan menjelang Natal.
Kini stok boneka Barbie


Summary dari Berita ID : 224 (hasil teks yang diringkas):

Transvision akan terus melakukan ekspansi untuk menghadirkan produk dan layanan berkualitas.Ayo pasang Transvision Hi-Speed sekarang juga! Jakarta, CNBC Indonesia - Internet merupakan salah satu kebutuhan utama masyarakat saat ini.
\Oringinal Article (Teks Asli):

Jakarta, CNBC Indonesia - Internet merupakan salah satu kebutuhan utama masyarakat saat ini.
Sebab, hampir seluruh kegiatan kini membutuhkan internet.Apalagi dalam tahun terakhir ini merupakan tahun yang menantang dan penuh dinamika bagi Indonesia akibat pandemi Covid-19.
Tidak dimungkiri pandemi juga membawa perubahan terhadap kebutuhan dan gaya hidup masyarakat terhadap tingkat adopsi digital yang meningkat.Untuk menjawab kebutuhan masyarakat tersebut, Transvision hadir dengan Transvision HI-Speed, sebuah layanan internet fiber unlimited hingga 100 Mbps.
Transvision Hi-Speed dilengkapi layanan TV kabel dengan pilihan channel live TV lengkap di antaranya 100+ Live T


Summary dari Berita ID : 227 (hasil teks yang diringkas):

Jakarta, CNBC Indonesia - Staples Center akan berganti nama menjadi Crypto.com Arena, di bawah perjanjian hak penamaan selama 20 tahun antara platform kripto dan AEG, perusahaan olahraga dan hiburan asal California, AS yang merupakan pemilik dan operator stadion tersebut. Logo baru tersebut akan diresmikan pada Hari Natal, ketika Los Angeles Lakers menjamu Brooklyn Nets. Dalam sebuah wawancara, CEO Crypto.com Kris Marszalek, seorang pengusaha kelahiran Polandia, mengatakan Crypto.com dan perusahaan induknya, Foris Technology telah menguntungkan tetapi tidak memberikan angka pasti. Perusahaan itu semula didirikan sebagai perusahaan rintisan bernama Monaco pada tahun 2016, sebagai bursa mata uang kripto yang juga menawarkan dompet digital dan kartu debit kripto.
\Oringinal Article (Teks Asli):

Jakarta, CNBC Indonesia - Staples Center akan berganti nama menjadi Crypto.com Arena, di bawah perjanjian hak penamaan selama 20 tahun a